In [1]:
import numpy as np
data = np.load('data.npz')
users = data['userlist'].astype(np.int)
movies = data['movielist'].astype(np.int)
ratings = (data['ratingmatrix'].astype(np.float)).T
moviegenre = data['genrematrix'].astype(np.int)
tagmatrix = data['tagmatrix'].astype(np.float)
tagmatrix = np.multiply(tagmatrix,100)

In [165]:
userprofilematrix = np.zeros((len(users), tagmatrix.shape[1]))
for i in range(len(users)):
    for j in range(len(movies)):
        if ratings[:,i][j] != -1:
            userprofilematrix[i] = np.add(userprofilematrix[i], tagmatrix[j])

IndexError: index 668 is out of bounds for axis 0 with size 668

In [3]:
from sklearn.utils.extmath import randomized_svd
U,s,V = randomized_svd(userprofilematrix, n_components=500,n_iter=20,random_state=None)

In [16]:
S = np.zeros((500,500))
np.fill_diagonal(S,s)

In [2]:
import scipy
from scipy.spatial import distance
def dist(u1, u2):
    c1,c2=[],[]
    for x,y in zip(u1,u2):
        if x!=-1.0 and y!=-1.0:
            c1.append(x)
            c2.append(y)
    r=0 if not c1 else distance.euclidean(c1,c2)
    return r

In [14]:
from sklearn import preprocessing 
user = 10
d=np.array([dist(ratings[user],y) for y in ratings])
d = 1-d/max(d)


In [35]:
from sklearn import preprocessing
user_neighbors=np.where(d>0.7)[0]
user_neighbors=user_neighbors[user_neighbors!=user]
neighbor_d = d[user_neighbors]
nonrated = np.where(ratings[user]==-1.0)[0]
nonrated_r = []
for i in nonrated:
    n = [x for x in user_neighbors if ratings[x][i]!=-1.0]
    n_d = d[n]
    if not n:
        nonrated_r.append(-1.0)
    else:
        rate = [ratings[x][i] for x in n]
        p = np.dot(rate, n_d) / sum(n_d)
        nonrated_r.append(p)
    
nonrated = [y for (x,y) in sorted(zip(nonrated_r,nonrated)) if x!=-1.0]
print(nonrated[:5])

[203, 251, 773, 1350, 1438]
